In [14]:
import json
from urllib.parse import urlparse

检查文件扩展名

在这一步骤，我们暂时丢弃所有非 HTML 文件的 URL

In [15]:
exts = {}

with open('./logs/urls-0402.log', 'r', encoding='utf-8') as f:
    log_lines = f.readlines()
for u in log_lines:
    ext = u.strip().split('.')[-1]
    # ext = urlparse(u).netloc
    if ext not in exts.keys():
        exts[ext] = 1
    else:
        exts[ext] = exts[ext] + 1

valid_exts = {}
for k, v in exts.items():
    if len(k) <= 8 and '/' not in k:
        valid_exts[k] = v

for k, v in sorted(valid_exts.items(), key=lambda x: x[1], reverse=True):
    print(f'{k}: {v}')


htm: 409081
html: 428
id)): 315
cn: 115
pptx: 96
jsp: 46
php: 41
com: 30
asp: 12
txt: 7
mht: 6
: 5
aspx: 4
shtml: 2
do: 2
id=1: 2
8b08051: 2
cer: 1
jspx: 1
gsp: 1
swf: 1
getUrl(: 1
GETUrl(: 1
dot: 1
wmv: 1
vcf: 1
null: 1


从日志中提取成功请求到内容的 URL

In [17]:
def get_urls_from_log(path):
    with open(path, 'r', encoding='utf-8') as f:
        log_lines = f.readlines()
    urls = []
    for u in log_lines:
        temp = u.split(':')
        if 'Succeed' in temp[2]:
            url = ':'.join(temp[3:]).strip()
            urls.append(url)
        # if 'Fail' in temp[2] and '404' not in temp[2]:
        #     print(u)
    print(f'load {len(urls)} urls from {path}')
    return urls

urls1 = get_urls_from_log('./logs/urls-0402.log')
# urls2 = get_urls_from_log('./logs/urls-0401.log')

load 375916 urls from ./logs/urls-0402.log


检查 URL 的 network location 部分

In [18]:
names = {}

for u in urls1:
    netloc = urlparse(u).netloc
    assert netloc != ''
    if netloc not in names.keys():
        names[netloc] = 1
    else:
        names[netloc] = names[netloc] + 1
    
    assert ('do=diff&rev' not in u) or ('howto.info.bit.edu.cn' in u)

for k, v in sorted(names.items(), key=lambda x: x[1], reverse=True):
    print(f'{k}: {v}')

www.bit.edu.cn: 81670
bit.edu.cn: 78984
mailto:ygcg@bit.edu.cn: 39492
xuxiangping@bit.edu.cn: 39492
sme.bit.edu.cn: 18948
ceep.bit.edu.cn: 11331
rw.bit.edu.cn: 7576
law.bit.edu.cn: 7538
grd.bit.edu.cn: 6610
me.bit.edu.cn: 6605
smen.bit.edu.cn: 6251
mse.bit.edu.cn: 4588
gonghui.bit.edu.cn: 4473
xuteli.bit.edu.cn: 4148
cs.bit.edu.cn: 3627
iidps.bit.edu.cn: 2794
kjc.bit.edu.cn: 2735
ls.bit.edu.cn: 2722
renshichu.bit.edu.cn: 2473
opt.bit.edu.cn: 2451
jxzx.bit.edu.cn: 2276
jwjc.bit.edu.cn: 2035
zzb.bit.edu.cn: 1958
isc.bit.edu.cn: 1944
sae.bit.edu.cn: 1806
abroad.bit.edu.cn: 1784
xyw.bit.edu.cn: 1763
celt.bit.edu.cn: 1613
student.bit.edu.cn: 1574
pdc.bit.edu.cn: 1278
tzb.bit.edu.cn: 1216
howto.info.bit.edu.cn: 1063
ttc.bit.edu.cn: 1046
xxgk.bit.edu.cn: 970
sie.bit.edu.cn: 927
international.bit.edu.cn: 883
english.bit.edu.cn: 821
cseem.bit.edu.cn: 745
kjpj.bit.edu.cn: 727
case.bit.edu.cn: 688
math.bit.edu.cn: 653
cce.bit.edu.cn: 614
zsc.bit.edu.cn: 547
ac.bit.edu.cn: 502
shock.bit.edu.cn: 49

In [ ]:
from url_normalize import url_normalize
bigset = set(urls1)
for u in urls2:
    u = url_normalize(u)
    if 'do=diff&rev' in u or '?rev=' in u:
        continue
    if u.endswith('.ai'):
        continue
    if u not in bigset:
        print(u)

这里发现了几个因为网络波动漏掉的:

https://ac.bit.edu.cn/szdw/jsml/znxxclykzyjs1/3fb0d053977a4308bf660af8d349c4d4.htm
https://ac.bit.edu.cn/szdw/jsml/znxxclykzyjs1/2416ed0621834ea681097f8e7c1b36bb.htm
https://ac.bit.edu.cn/szdw/jsml/znxxclykzyjs1/1a83997516ba4a7597e11082559c89ca.htm
https://ac.bit.edu.cn/szdw/jsml/znxxclykzyjs1/ee2dd81c83fe4941ac2e5fa54fc1096f.htm
https://ac.bit.edu.cn/szdw/jsml/znxxclykzyjs1/da6f8589ff77416690a6006bcf043ad7.htm

筛选：域名前面带"@"；URL 带文件扩展名

去重：http/https 重复； bit.edu.cn/www.bit.edu.cn 重复

补漏

In [19]:
unique_urls = set([
    'https://ac.bit.edu.cn/szdw/jsml/znxxclykzyjs1/3fb0d053977a4308bf660af8d349c4d4.htm',
    'https://ac.bit.edu.cn/szdw/jsml/znxxclykzyjs1/2416ed0621834ea681097f8e7c1b36bb.htm',
    'https://ac.bit.edu.cn/szdw/jsml/znxxclykzyjs1/1a83997516ba4a7597e11082559c89ca.htm',
    'https://ac.bit.edu.cn/szdw/jsml/znxxclykzyjs1/ee2dd81c83fe4941ac2e5fa54fc1096f.htm',
    'https://ac.bit.edu.cn/szdw/jsml/znxxclykzyjs1/da6f8589ff77416690a6006bcf043ad7.htm',
])
non_html_exts = set([
    'exe', 'apk', 'rar', 'zip', '7z', 'tgz',
    'png', 'jpeg', 'jpg', 'gif', 'mp3', 'mp4', 'mov', 'mpeg', 'avi', 'tif', '3gp', 'swf', 'wmv',
    'ai', 'dwg', 'vcf',
    'pptx', 'ppt', 'xls', 'xlsx', 'docx', 'doc', 'pdf', 'wps', 'pps', 'rtf','pptx',
    'eml', 'mht',
])
urls = get_urls_from_log('./logs/urls-0402.log')

for url in urls:
    parsed = urlparse(url)
    if '@' in parsed.netloc:
        continue
    if url.split('.')[-1].lower() in non_html_exts:
        continue
    if parsed.netloc == 'bit.edu.cn':
        parsed = parsed._replace(netloc='www.bit.edu.cn')
    url = parsed.geturl()
    unique_urls.add(url)
print(len(unique_urls))

filtered_urls = []
for url in unique_urls:
    # 协议去重: 若某一https开头的url 将其改为http后仍出现在集合里 那么这个url被忽略 否则加入列表
    parsed = urlparse(url)
    if parsed.scheme == 'https':
        parsed = parsed._replace(scheme='http')
        http_url = parsed.geturl()
        if http_url not in unique_urls:
            filtered_urls.append(http_url)
    else:
        filtered_urls.append(url)

for url in filtered_urls:
    assert url == url_normalize(url)
assert len(filtered_urls) == len(set(filtered_urls))
len(filtered_urls)

load 375916 urls from ./logs/urls-0402.log
217865


118219

导出为 json 格式

In [10]:
with open('./urls.json', 'w', encoding='utf-8') as f:
    json.dump(filtered_urls, f, indent=2)